In [18]:
import imaplib
import email
from email.header import decode_header

In [19]:
def get_mail_client(email_address):
    SMTP_SERVER = "imap.gmail.com"
    SMTP_PORT = 993
    
    password = ""
    with open("C:/Users/Owner/OneDrive/Documents/password.txt", "r") as f:
        password = f.read().strip()
        
    get_mail_client = imaplib.IMAP4_SSL(SMTP_SERVER,SMTP_PORT)
    get_mail_client.login(email_address, password)
    return get_mail_client

In [20]:
MAIL = get_mail_client('farooqui.awais@gmail.com')

In [4]:
MAIL.select('inbox')

('OK', [b'21948'])

In [ ]:
MAIL.select('"[Gmail]/All Mail"')

result, data = MAIL.uid('search', None, "ALL") # search all email and return uids
if result == 'OK':
    for num in data[0].split():
        result, data = MAIL.uid('fetch', num, '(RFC822)')
    if result == 'OK':
        email_message = email.message_from_bytes(data[0][1])    # raw email text including headers
        print('From:' + email_message['From'])

MAIL.close()
MAIL.logout()

In [4]:
INBOX = MAIL.select('INBOX')

In [5]:
print(INBOX)

('OK', [b'21906'])


In [28]:
SPAM = MAIL.select('[Gmail]/Spam')

In [6]:
def get_top_10_emails(category):
    # category can be 'Promotional, Updates or Forums
    # returns tuple
    status, response = MAIL.uid('search', 'X-GM-RAW "category:' + category + '"')

    # get email ids list
    response = response[0].decode('utf-8').split()
    response.reverse()
    response = response[:min(10, len(response))]
    return response

In [7]:
category = 'Promotional'

In [8]:
for uid in get_top_10_emails(category):
    status, data = MAIL.uid('fetch', uid, '(RFC822)')
    msg = get_msg_object(data)
    print(msg)

In [32]:
import email

def get_msg_object(data):
    for response_part in data:
        if isinstance(response_part, tuple):
            return email.message_from_bytes(response_part[1])

In [33]:
def get_email_ids(MAIL, label='INBOX', criteria='ALL', max_mails_to_look=10):
    MAIL.select(label)
    type, data = MAIL.search(None, criteria)
    mail_ids = data[0]
    id_list = mail_ids.split()
    # revers so that latest are at front
    id_list.reverse()
    id_list = id_list[: min(len(id_list), max_mails_to_look)]
    return id_list

In [34]:
def get_email_msg(email_id):
    email_id = str(int(email_id))
    type, data = MAIL.fetch(str(email_id), '(RFC822)')
    for response_part in data:
        if isinstance(response_part, tuple):
            return email.message_from_bytes(response_part[1])

In [35]:
# search in spam
MAIL.select('[Gmail]/Spam')
print("Looking in Spams")
id_list = get_email_ids(MAIL, "[Gmail]/Spam", max_mails_to_look=50)
for email_id in id_list:
    msg = get_email_msg(email_id)
    print(msg)

Looking in Spams
Delivered-To: farooqui.awais@gmail.com
Received: by 2002:a54:21c1:0:0:0:0:0 with SMTP id i1csp3123543eco;
        Mon, 9 May 2022 07:35:10 -0700 (PDT)
X-Google-Smtp-Source: ABdhPJxeHx14Zg07U4qQUkqy8KkPGMvMhhmaadE8m24nugRj/JJWDL5MXmkFvoptfx44njkx7pQd
X-Received: by 2002:ac2:5c4f:0:b0:472:30ed:e9aa with SMTP id s15-20020ac25c4f000000b0047230ede9aamr12481892lfp.123.1652106910254;
        Mon, 09 May 2022 07:35:10 -0700 (PDT)
ARC-Seal: i=1; a=rsa-sha256; t=1652106910; cv=none;
        d=google.com; s=arc-20160816;
        b=VQzpad3fNPqT83DgiEldqZ+W300v2c0h3y2qnXh9dGHIifFyeTvIDnWHShiEoxd8YD
         EE3qj4Oz2RVXrbvVgcB5+emI3uEu6ovqgHrT+iRZUWGmaC+qBos3XlFqDgdlIr5LfpXX
         CYrCwurXLOHFWuFicapn8M8TLHBZcfXZmqyVrzwoy9yn7Rdo4eTdD+qsyRmuWEVIG3S7
         VefU3jKVbMPEb7YZAAG/0FHbeeLuebaxvKfX4rk+SuhVxTnrEFgmDVKbwMUP8/YfVOQO
         ORnj9uMoVPO4kmPIGBQxkOADzMXpYQd3J3I5U9R4wRjkJlK/fybQkCEbPrHEJdTqxMdy
         rALw==
ARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [37]:
auth_results = msg.get("Authentication-Results", None)

In [38]:
print(auth_results)

mx.google.com;
       dkim=pass header.i=@truthout.org header.s=sl1 header.b=e8gwTmcR;
       spf=pass (google.com: domain of ebf293a7-dd7e-48b9-abfb-eb6b4067abdf@bounces.salsalabs.org designates 204.28.11.204 as permitted sender) smtp.mailfrom=ebf293a7-dd7e-48b9-abfb-eb6b4067abdf@bounces.salsalabs.org;
       dmarc=pass (p=NONE sp=NONE dis=NONE) header.from=truthout.org


In [39]:
def is_unsubscribe_present(msg):
    return "List-Unsubscribe" in msg.keys()

In [40]:
status, messages = MAIL.search(None, "ALL")

abort: socket error: [SSL: BAD_LENGTH] bad length (_ssl.c:2483)

In [ ]:
print(status)

In [ ]:
print(messages)

In [ ]:
# convert messages to a list of email IDs
messages = messages[0].split(b' ')

In [36]:
for mail in messages:
    _, msg = MAIL.fetch(mail, "(RFC822)")
    # mark the mail as deleted
MAIL.store(mail, "+FLAGS", "\\Deleted")

abort: socket error: [WinError 10054] An existing connection was forcibly closed by the remote host

In [22]:
# permanently remove mails that are marked as deleted
# from the selected mailbox (in this case, INBOX)
MAIL.expunge()
MAIL.close()
MAIL.logout()

('BYE', [b'LOGOUT Requested'])